<a href="https://colab.research.google.com/github/YasminaElkhazen/Synthetic_Data/blob/main/Synthetic_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The goal is to implement a content-based competition recommendation system depending on user's profile and skills.



##First Let's synthetize some data for a better model:


Importing existing data


In [17]:

import numpy as np
import pandas as pd
challenges = pd.read_csv('testchallenge.csv')
skills = pd.read_csv('skills.csv')

In [18]:
print(challenges.shape)
print(skills.shape)

(124, 31)
(8, 5)


In [19]:
challenges.head()

,_id/$oid,title,description,totalStars/$numberInt,starNumber/$numberInt,start,verified,objective,category/0/$oid,RecommendedSkills/0/$oid,...,users/0/registrationDate/$date/$numberLong,users/0/review,users/0/_id/$oid,company/$oid,category/1/$oid,submissions/0/$oid,users/0/submissionDate/$date/$numberLong,deadline/$date/$numberLong,RecommendedSkills/2/$oid,RecommendedSkills/3/$oid
0,65de752d9c5c71b1c8fd3e7f,Beyond Boundaries: Designing Inclusive Environ...,This design challenge calls for creating space...,0,0,False,True,Freelance,6565dbc91b552b582750a569,656dca46f0780635fb101703,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,65df31e09c5c71b1c8fd47f6,Logo Design Challenge,Unleash your creativity by designing a captiva...,0,0,False,True,Freelance,6565dbc91b552b582750a569,6565dd201b552b582750a78f,...,1.709144e+12,False,65df79269c5c71b1c8fd5320,6581ca220048a6a683135dbc,NaN,NaN,NaN,NaN,NaN,NaN
2,65df33bf9c5c71b1c8fd497c,Sustainable Minds²,Hackathon,0,0,True,True,Freelance,6565dbc11b552b582750a564,6565ddc01b552b582750a834,...,1.709127e+12,True,65df342b9c5c71b1c8fd4a58,65df335f9c5c71b1c8fd487d,6565dbc91b552b582750a569,65df34c89c5c71b1c8fd4aca,1.709127e+12,1.709130e+12,NaN,NaN
3,65de763a9c5c71b1c8fd402f,Interactive Recipe Finder,Create a web application that allows users to ...,0,0,False,True,Internship,6565dbc11b552b582750a564,6565dd7f1b552b582750a7a5,...,1.709118e+12,False,65df130d9c5c71b1c8fd4397,6581ca220048a6a683135dbc,NaN,NaN,NaN,NaN,657491f49f5ec2a17cfe3daa,65748f77274b04193293957e
4,65de752d9c5c71b1c8fd3e7f,UX Design Challenge: Redesigning a Travel App,Revamp an existing travel app's user experienc...,0,0,False,True,Freelance,6565dbc91b552b582750a569,656dca46f0780635fb101703,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
skills.head()

,_id,name,createdAt,updatedAt,__v
0,6565dd191b552b582750a784,Python,1701174553546,1701174771032,0
1,6565dd201b552b582750a78f,Java,1701174560081,1701174560081,0
2,6565dd891b552b582750a7bb,Ruby,1701174665059,1701174665059,0
3,6565dda61b552b582750a7fd,Responsive Design,1701174694679,1701174694679,0
4,6565ddc01b552b582750a834,UI/UX Design,1701174744574,1701174744574,0


In [21]:
# Dictionary mapping original column names to readable names
column_mapping = {
    '_id/$oid': 'Challenge_ID',
    'title': 'Title',
    'description': 'Description',
    'totalStars/$numberInt': 'Total_Stars',
    'starNumber/$numberInt': 'Star_Number',
    'start': 'Start_Date',
    'verified': 'Verified',
    'objective': 'Objective',
    'category/0/$oid': 'Category_ID',
    'RecommendedSkills/0/$oid': 'Recommended_Skill_1',
    'price/$numberInt': 'Price',
    'paid': 'Paid',
    'YoutubeLink': 'Youtube_Link',
    'nbruser/$numberInt': 'Number_of_Users',
    'owner/$oid': 'Owner_ID',
    'createdAt/$date/$numberLong': 'Created_At',
    'updatedAt/$date/$numberLong': 'Updated_At',
    '__v/$numberInt': 'Version',
    'paymentId': 'Payment_ID',
    'RecommendedSkills/1/$oid': 'Recommended_Skill_2',
    'users/0/user/$oid': 'User_ID',
    'users/0/registrationDate/$date/$numberLong': 'User_Registration_Date',
    'users/0/review': 'User_Review',
    'users/0/_id/$oid': 'User_Record_ID',
    'company/$oid': 'Company_ID',
    'category/1/$oid': 'Secondary_Category_ID',
    'submissions/0/$oid': 'Submission_ID',
    'users/0/submissionDate/$date/$numberLong': 'User_Submission_Date',
    'deadline/$date/$numberLong': 'Deadline',
    'RecommendedSkills/2/$oid': 'Recommended_Skill_3',
    'RecommendedSkills/3/$oid': 'Recommended_Skill_4'
}

# Rename columns
challenges = challenges.rename(columns=column_mapping)
challenges.head()

,Challenge_ID,Title,Description,Total_Stars,Star_Number,Start_Date,Verified,Objective,Category_ID,Recommended_Skill_1,...,User_Registration_Date,User_Review,User_Record_ID,Company_ID,Secondary_Category_ID,Submission_ID,User_Submission_Date,Deadline,Recommended_Skill_3,Recommended_Skill_4
0,65de752d9c5c71b1c8fd3e7f,Beyond Boundaries: Designing Inclusive Environ...,This design challenge calls for creating space...,0,0,False,True,Freelance,6565dbc91b552b582750a569,656dca46f0780635fb101703,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,65df31e09c5c71b1c8fd47f6,Logo Design Challenge,Unleash your creativity by designing a captiva...,0,0,False,True,Freelance,6565dbc91b552b582750a569,6565dd201b552b582750a78f,...,1.709144e+12,False,65df79269c5c71b1c8fd5320,6581ca220048a6a683135dbc,NaN,NaN,NaN,NaN,NaN,NaN
2,65df33bf9c5c71b1c8fd497c,Sustainable Minds²,Hackathon,0,0,True,True,Freelance,6565dbc11b552b582750a564,6565ddc01b552b582750a834,...,1.709127e+12,True,65df342b9c5c71b1c8fd4a58,65df335f9c5c71b1c8fd487d,6565dbc91b552b582750a569,65df34c89c5c71b1c8fd4aca,1.709127e+12,1.709130e+12,NaN,NaN
3,65de763a9c5c71b1c8fd402f,Interactive Recipe Finder,Create a web application that allows users to ...,0,0,False,True,Internship,6565dbc11b552b582750a564,6565dd7f1b552b582750a7a5,...,1.709118e+12,False,65df130d9c5c71b1c8fd4397,6581ca220048a6a683135dbc,NaN,NaN,NaN,NaN,657491f49f5ec2a17cfe3daa,65748f77274b04193293957e
4,65de752d9c5c71b1c8fd3e7f,UX Design Challenge: Redesigning a Travel App,Revamp an existing travel app's user experienc...,0,0,False,True,Freelance,6565dbc91b552b582750a569,656dca46f0780635fb101703,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
challenges.dropna().drop_duplicates()
challenges.shape

(124, 31)

In [23]:
print(challenges.groupby('Company_ID').count())


                          Challenge_ID  Title  Description  Total_Stars  \
Company_ID                                                                
6581ca220048a6a683135dbc            61     61           61           61   
65df335f9c5c71b1c8fd487d            31     31           31           31   

                          Star_Number  Start_Date  Verified  Objective  \
Company_ID                                                               
6581ca220048a6a683135dbc           61          61        61         61   
65df335f9c5c71b1c8fd487d           31          31        31         31   

                          Category_ID  Recommended_Skill_1  ...  User_ID  \
Company_ID                                                  ...            
6581ca220048a6a683135dbc           61                   61  ...       61   
65df335f9c5c71b1c8fd487d           31                   31  ...       31   

                          User_Registration_Date  User_Review  User_Record_ID  \
Company_ID      

In [24]:
challenges.groupby('User_ID').count()

,Challenge_ID,Title,Description,Total_Stars,Star_Number,Start_Date,Verified,Objective,Category_ID,Recommended_Skill_1,...,User_Registration_Date,User_Review,User_Record_ID,Company_ID,Secondary_Category_ID,Submission_ID,User_Submission_Date,Deadline,Recommended_Skill_3,Recommended_Skill_4
User_ID,,,,,,,,,,,,,,,,,,,,,
65679947ceb352bf75d7bf63,31,31,31,31,31,31,31,31,31,31,...,31,31,31,31,0,0,0,0,0,0
65df12d69c5c71b1c8fd4339,31,31,31,31,31,31,31,31,31,31,...,31,31,31,30,0,0,0,0,30,30
65df34199c5c71b1c8fd4a2a,31,31,31,31,31,31,31,31,31,31,...,31,31,31,31,31,31,31,31,0,0


##So as we can see there isn't enough data for the problem, in the next section we will try to merge different Kaggle datasets.

The following dataset contains different professional skills that a user may include in their profile:

In [25]:
skillset =pd.read_excel('/content/job_applicant_skills_list.xlsx')
skillset.shape

(252, 29)

In [26]:
skillset.head()

,user_id,skill 1,skill2,skill3,skill 4,skill 5,skill 6,skill 7,skill 8,skill 9,...,skill 19,skill 20,skill 21,skill 22,skill 23,skill 24,skill 25,skill 26,skill 27,skill 28
0,2,skill_collaboration,skill_dos,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,skill_cloud computing,skill_deep learning,skill_hadoop,skill_mentoring,skill_renewable energy,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,skill_collaboration,skill_deep learning,skill_mentoring,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,skill_insights,skill_windows,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,skill_attention to detail,skill_cloud computing,skill_coaching,skill_coding,skill_collaboration,skill_deep learning,skill_design patterns,skill_eclipse,skill_hadoop,...,skill_time management,skill_training,skill_web analytics,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
skillset = skillset.drop(columns=['user_id'])

In [28]:
clean_skillset = skillset.stack().reset_index(drop=True).to_frame(name='clean_skillset')
clean_skillset.shape

(1111, 1)

In [29]:
clean_skillset.dropna().drop_duplicates()
clean_skillset = pd.Series([x[6:] for x in clean_skillset['clean_skillset']])
clean_skillset.shape

(1111,)

In [30]:
clean_skillset.head(10)

0       collaboration
1                 dos
2     cloud computing
3       deep learning
4              hadoop
5           mentoring
6    renewable energy
7       collaboration
8       deep learning
9           mentoring
dtype: object

I will use the following dataset to generate challenges and their description:

In [31]:
internship=pd.read_csv('internship.csv')

In [32]:
internship.head()

,internship_title,company_name,location,start_date,duration,stipend
0,Java Development,SunbaseData,Work From Home,Immediately,6 Months,"₹ 30,000 /month"
1,Accounting and Finance,DAKSM & Co. LLP,Noida,Immediately,6 Months,"₹ 5,000-10,000 /month"
2,Sales & Digital Marketing,Bharat Natural Elements Private Limited,Bangalore,Immediately,6 Months,"₹ 5,000 /month"
3,Social Entrepreneurship,Hamari Pahchan NGO,Work From Home,Immediately,6 Months,Unpaid
4,Videography & Photography,Esquare Lifestyle,Bangalore,Immediately,6 Months,"₹ 12,000 /month"


In [33]:
internship.shape

(6485, 6)

In [34]:
internship=internship['internship_title'].drop_duplicates().dropna()

In [35]:
internship.shape


(2162,)

In [36]:
internship.to_csv('/content/internship_new.csv', index=False)


In [37]:
clean_skillset=clean_skillset.drop_duplicates()

In [38]:
clean_skillset.shape

(137,)

In [39]:
pip install pandas transformers sentence-transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 1.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

Cosine similarity to match jobs with the relebvant skills:

In [40]:
import pandas as pd
from sentence_transformers import SentenceTransformer, util

# Load the job titles and skills datasets (assuming they are already in pandas Series format)
job_titles = internship  # Replace with your actual Series containing job titles
skills = clean_skillset  # Replace with your actual Series containing skills

# Initialize the SentenceTransformer model
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

# Compute embeddings for job titles and skills
job_titles_embeddings = model.encode(job_titles.tolist(), convert_to_tensor=True)
skills_embeddings = model.encode(skills.tolist(), convert_to_tensor=True)

# Function to get the top 7 relevant skills for a job title based on cosine similarity
def get_top_skills(job_title_embedding, skills_embeddings, skills, top_n=7):
    cos_similarities = util.pytorch_cos_sim(job_title_embedding, skills_embeddings)[0]
    top_skills_indices = cos_similarities.topk(top_n).indices
    return [skills[i] for i in top_skills_indices]

# Add relevant skills to each job title
relevant_skills = []
for job_title_embedding in job_titles_embeddings:
    top_skills = get_top_skills(job_title_embedding, skills_embeddings, skills.tolist())
    relevant_skills.append(top_skills)

# Create a DataFrame with job titles and their relevant skills
df = pd.DataFrame({'Job Title': job_titles, 'Relevant Skills': relevant_skills})

# Print the updated DataFrame
print(df)


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.73k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

                              Job Title  \
0                      Java Development   
1                Accounting and Finance   
2             Sales & Digital Marketing   
3               Social Entrepreneurship   
4             Videography & Photography   
...                                 ...   
6450                Helpdesk Operations   
6465  Python/Django & React Engineering   
6467               Clients Relationship   
6472                    Product Testing   
6479                Field Sales Trainee   

                                        Relevant Skills  
0     [web development, software development, projec...  
1     [organizational, data management, data analysi...  
2     [digital marketing, content marketing, social ...  
3     [innovation, social media marketing, organizat...  
4     [graphic design, content marketing, presentati...  
...                                                 ...  
6450  [processing, solution development, troubleshoo...  
6465  [python, prog

In [41]:
df.shape

(2162, 2)

In [42]:
df.head()

,Job Title,Relevant Skills
0,Java Development,"[web development, software development, projec..."
1,Accounting and Finance,"[organizational, data management, data analysi..."
2,Sales & Digital Marketing,"[digital marketing, content marketing, social ..."
3,Social Entrepreneurship,"[innovation, social media marketing, organizat..."
4,Videography & Photography,"[graphic design, content marketing, presentati..."


In [43]:
from google.colab import files
df.to_csv('job_titles_with_skills_f.csv', index=False)

# Download the file
files.download('job_titles_with_skills_f.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

let's generate profile descriptions for our users:

In [44]:
!pip install transformers


In [45]:
sample=df.sample(n=200, random_state=42)


In [46]:
#_______________________________________test________________________________________________
from transformers import pipeline, set_seed

# Initialize the text generation pipeline with the GPT-2 model
generator = pipeline('text-generation', model='gpt2')

# Set a seed for reproducibility
set_seed(42)

# Define the input prompt for generating profile descriptions
prompt = "I'm a data scientist with 5 years of experience in machine learning and data analysis. My key skills include"

# Generate multiple sequences of profile descriptions
descriptions = generator(prompt, max_length=100, num_return_sequences=7)

# Print the generated profile descriptions
for i, description in enumerate(descriptions):
    print(f"Profile {i+1}: {description['generated_text']}")


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Profile 1: I'm a data scientist with 5 years of experience in machine learning and data analysis. My key skills include:

Performing data-mining exercises using a C++ compiler with SQL Server, OpenSUSE, and OpenShift

Solving complex numerical equations

Automating and analyzing large numbers by combining the data in a spreadsheet

Making mathematical calculations in SQL databases using SQL Server

What's in a Computer? It's all there at the end with a very clear definition
Profile 2: I'm a data scientist with 5 years of experience in machine learning and data analysis. My key skills include running a huge open source and open source R programming language with many features that you already understand.

My role is to help build and understand the data, and build applications from it.

My work will not be based solely on theory and practice, but I will try to get you to understand the algorithms that will be tested in this space.

There are other open source projects
Profile 3: I'm a d

In [47]:
# Imports
import pandas as pd
import os
from transformers import pipeline, set_seed
from google.colab import files
# Initialize the text generation pipeline with the GPT-2 model
generator = pipeline('text-generation', model='gpt2')

# Set a seed for reproducibility
set_seed(42)

# Load the dataframe
df = pd.read_csv('job_titles_with_skills_f.csv')

# Function to generate competition details
def generate_user_details(job_title, skills):
    prompt = f"I am a {job_title} with skills in {', '.join(skills)}. My key responsibilities include"
    generated_text = generator(prompt, max_length=500, num_return_sequences=1, truncation=True)
    return generated_text[0]['generated_text'].replace(prompt, '')

# Checkpointing function to save progress
def save_checkpoint(df, checkpoint_path):
    df.to_csv(checkpoint_path, index=False)
    print(f"Checkpoint saved to {checkpoint_path}")

# Path to save the checkpoint
checkpoint_path = "user_details_checkpoint.csv"

# Load from checkpoint if it exists
if os.path.exists(checkpoint_path):
    df = pd.read_csv(checkpoint_path)

# Process each row and save periodically
for idx, row in df.iterrows():
    if pd.isnull(row.get('user_details', None)):  # Check if the row has been processed
        df.at[idx, 'user_details'] = generate_user_details(row['Job Title'], row['Relevant Skills'])

        # Save checkpoint every 20 iterations
        if (idx + 1) % 10 == 0:
            save_checkpoint(df, checkpoint_path)
            df.to_csv("user_details_checkpoint.csv", index=False)

# Download the file

# Final save
files.download("df.csv")
save_checkpoint(df, checkpoint_path)
files.download("user_details_checkpoint.csv")

# Print the updated DataFrame
print(df.head())



Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Checkpoint saved to user_details_checkpoint.csv


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Checkpoint saved to user_details_checkpoint.csv


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...

Checkpoint saved to user_details_checkpoint.csv


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Checkpoint saved to user_details_checkpoint.csv


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


KeyboardInterrupt: 

In [48]:

save_checkpoint(df, checkpoint_path)
files.download("user_details_checkpoint.csv")


Checkpoint saved to user_details_checkpoint.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
sample.head()

In [ ]:
from google.colab import files
sample.to_csv('sample.csv', index=False)

# Download the file
files.download('sample.csv')


let's now tackle the challenges dataset

using gpt-2


In [ ]:
#_________________________________________test______________________________________

from transformers import pipeline, set_seed
import pandas as pd

# Sample DataFrame for testing
data = {
    'Job Title': ['Web Developer', 'Data Scientist', 'Graphic Designer'],
    'Relevant Skills': [['HTML', 'CSS', 'JavaScript', 'React', 'Node.js', 'UI/UX Design', 'Responsive Design'],
                        ['Python', 'Machine Learning', 'Data Analysis', 'Statistics', 'SQL', 'R', 'Big Data'],
                        ['Adobe Photoshop', 'Illustrator', 'InDesign', 'Creative Cloud', 'Typography', 'Branding', 'Layout Design']]
}

sample = pd.DataFrame(data)

# Initialize the text generation pipeline with the GPT-2 model
generator = pipeline('text-generation', model='gpt2')

# Set a seed for reproducibility
set_seed(42)

# Generate creative challenge ideas for the subsample
descriptions = []

for index, row in sample.iterrows():
    job_title = row['Job Title']
    skills = ', '.join(row['Relevant Skills'])
    prompt = f"As a {job_title} with expertise in {skills}, we will challenge you to push the boundaries of what can be achieved in this field. The project is thee following"

    # Generate a creative challenge idea
    challenge = generator(prompt, max_length=300, num_return_sequences=1, temperature=1, top_p=0.9)

    # Select the generated challenge idea
    challenge_description = challenge[0]['generated_text']

    # Extract the generated text and remove the initial prompt part
    challenge_description = challenge_description.replace(prompt, "").strip()

    descriptions.append(challenge_description)

# Add the creative challenge ideas as a new column
sample['Creative Challenge'] = descriptions

# View the updated DataFrame
print(sample.head())  # Adjust the number of rows displayed as needed


In [ ]:
sample.head()

In [ ]:
from google.colab import files
sample.to_csv('sample.csv', index=False)

# Download the file

files.download('sample.csv')

not a very relevant result, let's try llama-2:


In [ ]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

# Sample DataFrame
data = {
    'Job Title': ['Web Developer', 'Data Scientist', 'Graphic Designer'],
    'Relevant Skills': [['HTML', 'CSS', 'JavaScript', 'React', 'Node.js', 'Git', 'UX/UI'],
                        ['Python', 'R', 'SQL', 'Machine Learning', 'Data Analysis', 'TensorFlow', 'Keras'],
                        ['Photoshop', 'Illustrator', 'InDesign', 'Typography', 'Color Theory', 'UX/UI', 'Branding']]
}
sample = pd.DataFrame(data)

# Load the LLaMA-2 tokenizer and model with your access token
model_name = "meta-llama/Meta-Llama-3-8B"
access_token = ""  # Replace with your actual access token
tokenizer = AutoTokenizer.from_pretrained(model_name, token=access_token)
model = AutoModelForCausalLM.from_pretrained(model_name, token=access_token)

# Create a text generation pipeline with LLaMA-2
generator = pipeline('text-generation', model=model, tokenizer=tokenizer)

# Function to generate a competition prompt using LLaMA-2
def generate_competition_prompt(job_title, skills):
    skills_list = ', '.join(skills)
    prompt = (
        f"Make a comprehensive competition or hackathon for {job_title}s with skills in {skills_list}. "
        f"Here’s a detailed description of the competition, including its objectives and prizes. "
        f"Make it engaging and inspiring, appealing to participants in the field. "
        f"Be sure to include specific tasks or projects that participants will need to complete. "
        f"Outline the submission requirements and evaluation criteria."
    )
    competition = generator(prompt, max_length=300, num_return_sequences=1, temperature=1.2, top_p=0.95)
    return competition[0]['generated_text']

# Apply the function to each row in the DataFrame
sample['Competition'] = sample.apply(lambda row: generate_competition_prompt(row['Job Title'], row['Relevant Skills']), axis=1)

# Display the updated DataFrame
print(sample)


Let's now generate competitions for a each audience

In [ ]:
!pip install --upgrade huggingface_hub

In [ ]:
from huggingface_hub import login
login()

Get the model config, this will allow you to see the vocab, embeddings length etc


In [ ]:
from transformers import AutoConfig

# get the config
model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
config = AutoConfig.from_pretrained(model_name)

# print the config
print(config)


Transformers libary contains the predefined architecture (petrained model) which we can configure

In [ ]:
from transformers import AutoModelForCausalLM

# get the model
model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
model = AutoModelForCausalLM.from_pretrained(model_name)

# print details of the model
print(model)

In [ ]:
from huggingface_hub import login
login()

This allows you to print out details of the tokenizer for the model

In [ ]:
from transformers import AutoTokenizer

# get the tokenizer
model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# print the tokenizer
print(tokenizer)

Using inference:

In [ ]:
# imports
from transformers import LlamaTokenizer, pipeline, AutoModelForCausalLM

# set the model as tiny llama
model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

# get the tokenizer from the model
tokenizer = LlamaTokenizer.from_pretrained(model_name)

# Load the model
model = AutoModelForCausalLM.from_pretrained(model_name)

# Create a pipeline for text generation
generator = pipeline('text-generation', model=model, tokenizer=tokenizer)

# Generate text based on a prompt
prompt = "Who is Ada Lovelace?"
generated_text = generator(prompt, max_length=50)

# print the result
print(generated_text[0]['generated_text'])

In [ ]:
import pandas as pd
data= pd.read_csv("job_titles_with_skills_f.csv").sample(n=1000)


In [ ]:
# Imports
from transformers import LlamaTokenizer, pipeline, AutoModelForCausalLM
import pandas as pd
import os

# Set the model as TinyLlama
model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

# Get the tokenizer from the model
tokenizer = LlamaTokenizer.from_pretrained(model_name)

# Load the model
model = AutoModelForCausalLM.from_pretrained(model_name)

# Create a pipeline for text generation
generator = pipeline('text-generation', model=model, tokenizer=tokenizer)


# Convert to DataFrame
df = pd.DataFrame(data)

# Function to generate competition details
def generate_competition_details(job_title, skills):
    prompt = f"You are a problem-setter, your mission is to create a competition for {job_title} with skills in {skills}. Provide the following details: problem description, awards, evaluation criteria."
    generated_text = generator(prompt, max_length=300)
    return generated_text[0]['generated_text'].replace(prompt, '')

# Checkpointing function to save progress
def save_checkpoint(df, checkpoint_path):
    df.to_csv(checkpoint_path, index=False)
    print(f"Checkpoint saved to {checkpoint_path}")

# Path to save the checkpoint
checkpoint_path = "competition_details_checkpoint.csv"

# Load from checkpoint if it exists
if os.path.exists(checkpoint_path):
    df = pd.read_csv(checkpoint_path)

# Process each row and save periodically
for idx, row in df.iterrows():
    if pd.isnull(row.get('competition_details', None)):  # Check if the row has been processed
        df.at[idx, 'competition_details'] = generate_competition_details(row['Job Title'], row['Relevant Skills'])
        save_checkpoint(df, checkpoint_path)

# Final save
save_checkpoint(df, checkpoint_path)

# Print the updated DataFrame
print(df)



 Path to save the checkpoint


In [ ]:
import pandas as pd
users = pd.read_csv("users.csv").dropna()
challenges = pd.read_csv("challenges.csv").dropna()

In [ ]:
users.shape

In [ ]:
challenges.shape

In [ ]:
users.head()

In [ ]:
challenges.head()

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import random

# Sample user dataset
data = {
    "job_title": ["Data Scientist", "Web Developer", "AI Engineer"],
    "skills": ["Python, Machine Learning", "HTML, CSS, JavaScript", "TensorFlow, PyTorch"],
    "description": ["Expert in data science", "Experienced web developer", "Skilled AI engineer"]
}

# Convert to DataFrame
users = pd.DataFrame(data)

# Add an ID column to the users DataFrame
users['user_id'] = range(1, len(users) + 1)

# Add columns for submitted challenges and participated challenges
users['submitted_challenges'] = [[] for _ in range(len(users))]
users['participated_challenges'] = [[] for _ in range(len(users))]

# Create challenges DataFrame
challenges_data = []
start_date = datetime(2023, 1, 1)
end_date = datetime(2024, 12, 31)
delta = end_date - start_date

for _, row in users.iterrows():
    random_days = random.randint(0, delta.days)
    created_at = start_date + timedelta(days=random_days)
    duration = timedelta(days=np.random.randint(30, 90))
    end_at = created_at + duration
    updated_at = created_at + timedelta(days=np.random.randint(1, duration.days))

    challenges_data.append({
        "job_title": row['job_title'],
        "category": "Competition",
        "creation_date": created_at,
        "update_date": updated_at,
        "deadline": end_at
    })

challenges = pd.DataFrame(challenges_data)
challenges['challenge_id'] = range(1, len(challenges) + 1)

# Add challenges participated to users DataFrame based on job title
for _, user_row in users.iterrows():
    participated_challenge_ids = challenges.sample(n=np.random.randint(1, 4))['challenge_id'].tolist()
    users.at[user_row.name, 'participated_challenges'] = participated_challenge_ids

# Add a column for submitted challenges to users DataFrame based on participated challenges
for _, user_row in users.iterrows():
    if len(user_row['participated_challenges']) > 0:
        num_submitted = np.random.randint(1, len(user_row['participated_challenges']) + 1)
        submitted_challenge_ids = random.sample(user_row['participated_challenges'], num_submitted)
        users.at[user_row.name, 'submitted_challenges'] = submitted_challenge_ids

# Print the updated DataFrames
print("Users DataFrame:")
print(users)
print("\nChallenges DataFrame:")
print(challenges)


In [49]:

import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import random


# Convert to DataFrame
users = pd.read_csv("/content/user_details_checkpoint.csv").head(1100)
challenges = pd.read_csv("/content/job_titles_with_skills_f.csv").head(200)
# Add an ID column to the users DataFrame
users['user_id'] = range(1, len(users) + 1)

# Add columns for submitted challenges and participated challenges
users['submitted_challenges'] = [[] for _ in range(len(users))]
users['participated_challenges'] = [[] for _ in range(len(users))]

# Create challenges DataFrame

start_date = datetime(2023, 1, 1)
end_date = datetime(2024, 12, 31)
delta = end_date - start_date
challenges_data = []
for _, row in users.iterrows():
    random_days = random.randint(0, delta.days)
    created_at = start_date + timedelta(days=random_days)
    duration = timedelta(days=np.random.randint(30, 90))
    end_at = created_at + duration
    updated_at = created_at + timedelta(days=np.random.randint(1, duration.days))

    challenges_data.append({

        "creation_date": created_at,
        "update_date": updated_at,
        "deadline": end_at
    })

new_challenges = pd.DataFrame(challenges_data)
challenges = pd.concat([challenges, new_challenges], ignore_index=True , axis=1)

challenges['challenge_id'] = range(1, len(challenges) + 1)

# Add challenges participated to users DataFrame based on job title
for _, user_row in users.iterrows():
        sample_size =  np.random.randint(1, 4)
        participated_challenge_ids = challenges.sample(n=np.random.randint(1, 4))['challenge_id'].tolist()
        users.at[user_row.name, 'participated_challenges'] = participated_challenge_ids

# Add a column for submitted challenges to users DataFrame based on participated challenges
for _, user_row in users.iterrows():
    if len(user_row['participated_challenges']) > 0:
        num_submitted = np.random.randint(1, len(user_row['participated_challenges']) + 1)
        submitted_challenge_ids = random.sample(user_row['participated_challenges'], num_submitted)
        users.at[user_row.name, 'submitted_challenges'] = submitted_challenge_ids

# Print the updated DataFrames
print("Users DataFrame:")
print(users)
print("\nChallenges DataFrame:")
print(challenges)

Users DataFrame:
                      Job Title  \
0              Java Development   
1        Accounting and Finance   
2     Sales & Digital Marketing   
3       Social Entrepreneurship   
4     Videography & Photography   
...                         ...   
1095    Foreign Exchange Market   
1096            C++ Development   
1097         Sales Coordination   
1098  Doubt Solving (Chemistry)   
1099     AI Fullstack Developer   

                                        Relevant Skills  \
0     ['web development', 'software development', 'p...   
1     ['organizational', 'data management', 'data an...   
2     ['digital marketing', 'content marketing', 'so...   
3     ['innovation', 'social media marketing', 'orga...   
4     ['graphic design', 'content marketing', 'prese...   
...                                                 ...   
1095  ['e-commerce', 'digital marketing', 'social me...   
1096  ['software development', 'web development', 's...   
1097  ['organizational', 'colla

In [50]:
challenges.dropna()
challenges.head()

,0,1,2,3,4,challenge_id
0,Java Development,"['web development', 'software development', 'p...",2024-10-16,2024-12-07,2024-12-23,1
1,Accounting and Finance,"['organizational', 'data management', 'data an...",2023-04-25,2023-05-10,2023-06-22,2
2,Sales & Digital Marketing,"['digital marketing', 'content marketing', 'so...",2023-01-26,2023-03-28,2023-04-08,3
3,Social Entrepreneurship,"['innovation', 'social media marketing', 'orga...",2023-10-09,2023-11-17,2023-11-28,4
4,Videography & Photography,"['graphic design', 'content marketing', 'prese...",2023-09-08,2023-11-30,2023-12-04,5


In [51]:
challenges.shape

(1100, 6)

In [55]:
import pandas as pd

# Assuming you have the users and challenges datasets as dataframes
users_df = pd.DataFrame(users)  # Replace users_data with your actual data
challenges_df = pd.DataFrame(challenges)  # Replace challenges_data with your actual data

# Specify the paths where you want to save the datasets
users_path = 'users_dataset.csv'
challenges_path = 'challenges_dataset.csv'

# Save the dataframes to CSV files
users_df.to_csv(users_path, index=False)
challenges_df.to_csv(challenges_path, index=False)

print(f'Users dataset saved to {users_path}')
print(f'Challenges dataset saved to {challenges_path}')
from google.colab import files

# Download the users dataset
files.download(users_path)

# Download the challenges dataset
files.download(challenges_path)


Users dataset saved to users_dataset.csv
Challenges dataset saved to challenges_dataset.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [52]:
users.dropna()
users.shape

(1100, 6)

In [53]:
users.head()

,Job Title,Relevant Skills,user_details,user_id,submitted_challenges,participated_challenges
0,Java Development,"['web development', 'software development', 'p...",": Writing code, implementing client side proto...",1,"[506, 882]","[91, 506, 882]"
1,Accounting and Finance,"['organizational', 'data management', 'data an...",": [ t:i, g:a, u:z ], [ a:p:t :], [ i:n :], [ a...",2,[885],"[1011, 144, 885]"
2,Sales & Digital Marketing,"['digital marketing', 'content marketing', 'so...",": designing product concepts, integrating key ...",3,[709],"[308, 709]"
3,Social Entrepreneurship,"['innovation', 'social media marketing', 'orga...",": writing the paper, writing up the code, revi...",4,[449],[449]
4,Videography & Photography,"['graphic design', 'content marketing', 'prese...",writing video for our blog of the day and cov...,5,[6],[6]
